In [ ]:
print("foo")

foo


In [ ]:
import requests
# ───── Google Slides 類似検索 API ─────────────────────
from googleapiclient.discovery import build          # type: ignore
from google.oauth2.credentials import Credentials    # type: ignore
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from langchain_community.vectorstores import Chroma     # type: ignore
from langchain.schema import Document                   # type: ignore
from langchain_openai import OpenAIEmbeddings            # type: ignore

from chromadb.config import Settings
import tempfile, os

PRESENTATION_ID = "1xW8Lze5bfwUzNd9ZqputgTFyQJdoKK3f3I7esGACAds"

def search():
    query = (request.get_json() or {}).get("query", "").strip()
    if not query:
        return jsonify({"error": "query_required"}), 400

    # 認証トークン
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as f:
            f.write(creds.to_json())

    service = build("slides", "v1", credentials=creds)
    presentation = service.presentations().get(
        presentationId=PRESENTATION_ID
    ).execute()

    # テキスト抽出
    slide_texts: list[str] = []
    for slide in presentation.get("slides", []):
        parts = []
        for elem in slide.get("pageElements", []):
            txt = (
                elem.get("shape", {})
                    .get("text", {})
                    .get("textElements", [])
            )
            for t in txt:
                run = t.get("textRun")
                if run:
                    parts.append(run.get("content", ""))
        slide_texts.append("".join(parts))

    # ベクトル検索
    docs = [
        Document(page_content=slide_texts[i], metadata={"idx": i})
        for i in range(len(slide_texts))
    ]

    store = Chroma.from_documents(
        docs,
        OpenAIEmbeddings(api_key=OPENAI_API_KEY),
        collection_name="slides",
        client_settings=Settings(
            chroma_db_impl="duckdb+parquet",
            persist_directory=TMP_DIR,
        ),
    )
    results = store.similarity_search_with_score(query, k=3)

    return jsonify([
        {
            "slide_index": doc.metadata["idx"] + 1,
            "content": doc.page_content,
            "score": score
        }
        for doc, score in results
    ])


ModuleNotFoundError: No module named 'request'

In [11]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv("../.env"))

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
SECRET_KEY     = os.getenv("FLASK_SECRET_KEY", "PLEASE_CHANGE_ME")

SCOPES = ["https://www.googleapis.com/auth/presentations.readonly"]
PRESENTATION_ID = "1xW8Lze5bfwUzNd9ZqputgTFyQJdoKK3f3I7esGACAds"

In [13]:
TMP_DIR = os.path.join(tempfile.gettempdir(), "chroma_slides")
os.makedirs(TMP_DIR, exist_ok=True)
# 認証トークン
creds = None
if os.path.exists("../app/token.json"):
    creds = Credentials.from_authorized_user_file("../app/token.json", SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=0)
    with open("token.json", "w") as f:
        f.write(creds.to_json())

service = build("slides", "v1", credentials=creds)
presentation = service.presentations().get(
    presentationId=PRESENTATION_ID
).execute()

# テキスト抽出
slide_texts: list[str] = []
for slide in presentation.get("slides", []):
    parts = []
    for elem in slide.get("pageElements", []):
        txt = (
            elem.get("shape", {})
                .get("text", {})
                .get("textElements", [])
        )
        for t in txt:
            run = t.get("textRun")
            if run:
                parts.append(run.get("content", ""))
    slide_texts.append("".join(parts))

# ベクトル検索
docs = [
    Document(page_content=slide_texts[i], metadata={"idx": i})
    for i in range(len(slide_texts))
]

store = Chroma.from_documents(
    docs,
    OpenAIEmbeddings(api_key=OPENAI_API_KEY),
    collection_name="slides",
    client_settings=Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=TMP_DIR,
    ),
)
results = store.similarity_search_with_score(query, k=3)


ValueError: [91mYou are using a deprecated configuration of Chroma.

[94mIf you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/deployment/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/deployment/migration for more information or join our discord at https://discord.gg/MMeYNTmh3x for help![0m